In [1]:
import datetime
from os.path import join

from script.core.transformers import *
from script.models.cnn import CNN
from script.core.models import *
from script.core.dataset import Watcher
from script.core.utils import read_dataset, get_result, logging
from script.core.pipeline import Pipeline

# linear models
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# sklearn feachure extractors
from sklearn.feature_extraction.text import CountVectorizer as count
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.preprocessing import LabelEncoder

# Pure DilyaraModel
# from intent_classifier.intent_model.model_wrap import KerasMulticlassModel as DilyaraModel
# from script.models.cnn import CNN
# from script.models.dcnn import DCNN

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/mks/envs/intent_script/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
2018-04-03 18:30:49.209 DEBUG in 'matplotlib.backends'['__init__'] at line 90: backend module://ipykernel.pylab.backend_inline version unknown


# Read csv file and create Dataset

In [2]:
def init_dataset_tiny(file_path, language, dataset_name, date, seed=42):
    pure_data = read_dataset(file_path, True, True)  # It not default meanings!!!
    start_dataset = Watcher(pure_data, date, language, dataset_name,
                            seed=seed)  # classes_descriptions = {} we can do it

    ######################################################################################
    dataset = start_dataset.split([0.1, 0.1])
    data = dataset.data['test']
    start_dataset = Watcher(data, date, language, dataset_name, seed)
    ######################################################################################

    return start_dataset

date = datetime.datetime.now()
dataset_name = 'vkusvill'
language = 'russian'
file_path = join('./data', language, dataset_name, 'data', 'vkusvill_all_categories.csv')

dataset = init_dataset_tiny(file_path, language, dataset_name, date)

/home/mks/envs/intent_script/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Operations

In [ ]:
spl_conf = {'op_type': 'transformer',
            'name': 'Speller',
            'request_names': ['base'],
            'new_names': ['base'],
            'path': './DeepPavlov/deeppavlov/configs/error_model/brillmoore_kartaslov_ru.json'}

tok_conf = {'op_type': 'transformer',
            'name': 'Tokenizer',
            'request_names': ['base'],
            'new_names': ['base']}

lem_conf = {'op_type': 'transformer',
            'name': 'Lemmatizer',
            'request_names': ['base'],
            'new_names': ['base']}

concat = TextConcat()

tfidf_conf_1 = {'op_type': 'vectorizer', 'name': 'tf-idf vectorizer',
                'request_names': ['train', 'valid', 'test'], 'new_names': ['train_vec', 'valid_vec', 'test_vec']}
tfidf_conf_2 = {'op_type': 'vectorizer', 'name': 'count_vectorizer',
                'request_names': ['train', 'valid', 'test'], 'new_names': ['train_vec', 'valid_vec', 'test_vec']}
tfidf_ = sktransformer(tfidf, tfidf_conf_1)
count_ = sktransformer(count, tfidf_conf_2)

In [ ]:
dataset.data.keys()

In [3]:
dataset.data['base']['request'][0]

'в магазине 1018м мыт сукромка7 место выкладки теплые стеллажи нонфуд зафиксирована температура ниже 18 температура на начало дня 16 3температура в середине дня 16 4 температура на конец дня 16 1'

In [ ]:
config = get_cnn_conf(path_1, emb_path)
WCNN = GetCNN(CNN, config)
pipe = Pipeline([(Tokenizer,), (FasttextVectorizer,), (WCNN,)])
pipe.run(dataset)

In [4]:
dataset = dataset.load_data('38b2f8a46a974b45d9eaa3b76f304a60')
dataset.data.keys()

dict_keys(['base'])

In [5]:
dataset.data['base']['request'][0]

['в',
 'магазине',
 '1018м',
 'мыт',
 'сукромка7',
 'место',
 'выкладки',
 'теплые',
 'стеллажи',
 'нонфуд',
 'зафиксирована',
 'температура',
 'ниже',
 '18',
 'температура',
 'на',
 'начало',
 'дня',
 '16',
 '3температура',
 'в',
 'середине',
 'дня',
 '16',
 '4',
 'температура',
 'на',
 'конец',
 'дня',
 '16',
 '1']

In [ ]:
# s = dataset.data['base']['request'][0]
# s = s[1:-1]
# s = s.split(', ')
# s = [x[1:-1] for x in s]
# s

In [ ]:
sam = lambda s: [x[1:-1] for x in s[1:-1].split(', ')]

In [ ]:
dataset.data['base']['request'] = dataset.data['base']['request'].apply(sam)

In [ ]:
dataset.data['base']['request'][0]

In [7]:
config = get_cnn_conf(path_1, emb_path)
WCNN = GetCNN(CNN, config)
pipe = Pipeline([(FasttextVectorizer,), (WCNN,)])

In [8]:
pipe.run(dataset)

  5%|▍         | 181/3661 [00:00<00:01, 1802.40it/s]

[ Starting vectorization ... ]
[ Vectorization of train part of dataset ... ]


 43%|████▎     | 177/407 [00:00<00:00, 1765.61it/s]

[ Vectorization of valid part of dataset ... ]


 43%|████▎     | 173/407 [00:00<00:00, 1723.05it/s]

[ Vectorization of test part of dataset ... ]


100%|██████████| 407/407 [00:00<00:00, 1758.57it/s]


[ Vectorization was ended. ]
[ Initializing intent_model from scratch ]

____Training over 3661 samples____


train -->	updates: 1	loss: 0.358571857213974	fmeasure: 0.0	 
epochs_done: 1
train -->	updates: 59	loss: 0.23870240151882172	fmeasure: 0.42696624994277954	 
epochs_done: 2
train -->	updates: 117	loss: 0.19705097377300262	fmeasure: 0.5416666269302368	 
epochs_done: 3
train -->	updates: 175	loss: 0.18926946818828583	fmeasure: 0.54347825050354	 
epochs_done: 4
train -->	updates: 233	loss: 0.17898912727832794	fmeasure: 0.5154638886451721	 
epochs_done: 5


# Pipeline with Neural Models

## Run

In [6]:
class GetCNN(BaseModel):
    def init_model(self, dataset):
        classes = dataset.get_classes()
        classes = ' '.join([str(x) for x in classes])
        self.config['classes'] = classes
        
        super().init_model(dataset)
        
        return self

def get_cnn_conf(path, emb_path, fit_names=None, predict_names=None, new_names=None):
    with open(path, 'r') as conf:
        config = json.load(conf)

    config['op_type'] = 'model'
    config['name'] = 'cnn'
    
    if fit_names is not None:
        config['fit_names'] = fit_names
    else:
        config['fit_names'] = ['train_vec']
    
    if predict_names is not None:
        config['predict_names'] = predict_names
    else:
        config['predict_names'] = ['test_vec']
        
    if new_names is not None:
        config['new_names'] = new_names
    else:
        config['new_names'] = ['predicted_test']
    
    config['fasttext_model'] = emb_path
    
    return config

path_0 = './configs/models/CNN/CNN_opt.json'
path_1 = './configs/models/CNN.json'
emb_path = './data/russian/embeddings/ft_0.8.3_nltk_yalen_sg_300.bin'

In [ ]:
# config = get_cnn_conf(path_0, emb_path, fit_names=['train'], predict_names=['test'])
# dataset = init_dataset()
# Dilyara = GetCNN(DilyaraModel, config)
# neuro_pipe = [(Tokenizer, ), (Lemmatizer,), (TextConcat,), (Dilyara,), (GetResult,)]
# pipeline_0 = Pipeline(neuro_pipe, mode='infer', output='dataset')
# neurodata = pipeline_0.run(dataset)

In [ ]:
config = get_cnn_conf(path_1, emb_path)
WCNN = GetCNN(CNN, config)
neuro_pipe = [(Tokenizer, ), (FasttextVectorizer,), (WCNN,)]
pipeline_1 = Pipeline(neuro_pipe, mode='train', output=None)
neurodata = pipeline_1.run(dataset)

In [ ]:
model = pipeline_1.get_last_model()

In [ ]:
model = model[1]

In [ ]:
# pipe_ = [(Tokenizer, ), (FasttextVectorizer,)]
# pipe = Pipeline(pipe_)

# dataset = init_dataset_tiny(file_path, language, dataset_name, date)
data = model.predict(dataset, predict_name='test_vec', new_name='pred')

In [ ]:
data

In [ ]:
data.data.keys()

In [ ]:
pred_data.data['pred'][0].shape

In [ ]:
pred_data.data['predicted_test'][0][0]

In [ ]:
pred_data.data['predicted_test'][0][0].max()

In [ ]:
argmax = np.argmax(pred_data.data['predicted_test'][0], axis=1)

In [ ]:
argmax

In [ ]:
np.array(pred_data.data['test']['report'][:64])

In [ ]:
class GetResult(BaseTransformer):
    def __init__(self, config=None):
        if config is None:
            self.config = {'op_type': 'transformer',
                           'name': 'Resulter',
                           'request_names': ['predicted_test'],
                           'new_names': ['test']}
        else:
            self.config = config

        super().__init__(self.config)

    def _transform(self, dataset):
        request, report = dataset.main_names

        pred_name = self.config['request_names'][0]
        print(pred_name)
        real_name = self.config['new_names'][0]
        print(real_name)
        pred_data = dataset.data[pred_name]
        real_data = np.array(dataset.data[real_name][report])

        preds = pred_data[0]
        for x in pred_data[1:]:
            preds = np.concatenate((preds, x), axis=0)

        preds = np.argmax(preds, axis=1)
        for i, x in enumerate(preds):
            preds[i] = x + 1

        preds = preds[:len(real_data)]

        results = get_result(preds, real_data)
        dataset.data['results'] = results

        conf = dataset.pipeline_config
        date = dataset.date
        logging(results, conf, date)

        return dataset

end_pipe = Pipeline([(GetResult,)])
end_res = end_pipe.run(pred_data)

In [ ]:
end_res.data['results']

In [ ]:
# config = get_cnn_conf(path_1, emb_path)
# dataset = init_dataset()
# WCNN = GetCNN(CNN, config)
# neuro_pipe = [(Tokenizer, ), (FasttextVectorizer,), (WCNN,), (GetResult,)]
# pipeline_1 = Pipeline(neuro_pipe, mode='infer', output='dataset')
# neurodata = pipeline_1.run(dataset)

In [ ]:
# config = get_cnn_conf(path_1, emb_path)
# dataset = init_dataset()
# dCNN = GetCNN(DCNN, config)
# neuro_pipe = [(Tokenizer, ), (FasttextVectorizer,), (dCNN,), (GetResult,)]
# pipeline_2 = Pipeline(neuro_pipe, mode='infer', output='dataset')
# neurodata = pipeline_2.run(dataset)

# Pipelines with Linear Models

In [ ]:
conf_0 = {'op_type': 'model', 'name': 'Linear Regression',
          'fit_names': ['train_vec'], 'new_names': ['predicted_test'],
          'predict_names': ['test_vec']}

## Run pipeline with linear regression

In [ ]:
# (Tokenizer, tok_conf), (Lemmatizer,), (concat, None),
# dataset = init_dataset()
# LR = skmodel(LogisticRegression, conf_0)

# pipe_0 = [(tfidf_, tfidf_conf_2), (LR,), (GetResultLinear_W,)]
# pipeline_0 = Pipeline(pipe_0, mode='train', output=None)
# pipeline_1 = Pipeline(pipe_0, mode='infer', output='dataset')
# res = pipeline_1.run(dataset)

## Run pipeline with SVC

In [ ]:
# dataset = init_dataset()

# conf_0['name'] = 'LinearSVC'
# conf_0['op_type'] = 'model'
# LSVC = skmodel(LinearSVC, conf_0)

# # (Speller, spl_conf),

# pipe_1 = [(Tokenizer, ), (Lemmatizer,), (concat, None), (tfidf_, tfidf_conf_2),
#           (LSVC,), (GetResultLinear_W,)]
# pipeline_2 = Pipeline(pipe_1, mode='train', output=None)
# pipeline_3 = Pipeline(pipe_1, mode='infer', output='dataset')

# res = pipeline_3.run(dataset)

##  Run pipeline with Random Forest

In [ ]:
# dataset = init_dataset()

# conf_0['name'] = 'RandomForestClassifier'
# conf_0['op_type'] = 'model'
# RFC = skmodel(RandomForestClassifier, conf_0)


# # (Speller, spl_conf), 
# pipe_2 = [(Tokenizer, ), (Lemmatizer,), (concat, None), (tfidf_, tfidf_conf_2),
#           (RFC,), (GetResultLinear_W,)]
# pipeline_4 = Pipeline(pipe_2, mode='train', output=None)
# pipeline_5 = Pipeline(pipe_2, mode='infer', output='dataset')

# res = pipeline_5.run(dataset)

## Run pipeline with GBM

In [ ]:
# from os.path import join

# date = datetime.datetime.now()
# dataset_name = 'vkusvill'
# language = 'russian'
# file_path = join('./data', language, dataset_name, 'data', 'vkusvill_all_categories.csv')

# dataset = init_dataset_tiny(file_path, language, dataset_name, date)

# conf_0['name'] = 'LGBMClassifier'
# conf_0['op_type'] = 'model'
# LGBM = skmodel(LGBMClassifier, conf_0)

# # (Speller, spl_conf),
# pipe_3 = [(Tokenizer, ), (Lemmatizer,), (concat, None), (tfidf_, ),
#           (LGBM,), (GetResultLinear_W,)]
# pipeline_6 = Pipeline(pipe_3, mode='train', output=None)
# pipeline_7 = Pipeline(pipe_3, mode='infer', output='dataset')

# res = pipeline_7.run(dataset)